GRA2PES Demo
================

This notebook demonstrates multiple components of the GRA2PES framework, including:
- **Base data loading**: Loading the base data given the GRA2PES configuration.
- **Regridding**: Regridding the base LCC data to the target Lat/Lon grid. 
- **Regridded data loading**: Loading the regridded data given the GRA2PES configuration.
- **Basic processing**: Applying things like unit conversion etc. to the regridded data.
- **Emissions visualization**: Visualizing the regridded data using matplotlib.
- **Ratio visualization**: Visualizing the ratio of two regridded datasets.

### Setup
Ensure that the parameters in inventories/gra2pes/gra2pes_config.py are set correctly for your use case.

In [5]:
#Import necessary libraries
import os
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe
import sys 
import matplotlib.pyplot as plt
import pyproj
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
sys.path.append('..')
from inventories.gra2pes import gra2pes_utils , gra2pes_config, gra2pes_regrid
from utils import gen_utils, datetime_utils

# Base Data Loading/Examining
"Base" data referes to the data that is loaded from the dataset downloaded and organized by inventories/gra2pes/gra2pes_base_creator.py. These are the extracted tar.gz files from either the NOAA FTP or HTTPS servers. 

In [8]:
config = gra2pes_config.Gra2pesConfig() # Load the configuration 
extra_ids = 'methane' #define any "extra" ids that are not included in the default set of species
specs = ['CO2','CO','HC01']#,'HC02','HC14','NH3','NOX','SO2']

BGH = gra2pes_utils.BaseGra2pesHandler(config,specs = specs, extra_ids = extra_ids) #create the handler object

sector = 'AG'
year = 2021
month = 1
day_type = 'weekdy'
base_ds = BGH.load_fmt_fullday(sector, year, month, day_type, check_extra=False)
base_ds

<xarray.Dataset> Size: 8GB
Dimensions:   (utc_hour: 24, zlevel: 20, south_north: 1008, west_east: 1332)
Coordinates:
    XLAT      (south_north, west_east) float32 5MB 18.19 18.2 ... 52.42 52.41
    XLONG     (south_north, west_east) float32 5MB -121.8 -121.7 ... -58.58
  * utc_hour  (utc_hour) int64 192B 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Dimensions without coordinates: zlevel, south_north, west_east
Data variables:
    CO2       (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
    CO        (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
    HC01      (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
Attributes: (12/20)
    CEN_LON:        -97.84192
    MAP_PROJ_CHAR:  Lambert Conformal
    TITLE:          GRA2PESv1.0 Emissions Dataset
    TRUELAT1:       33.0
    DX:             4000.0
    MOAD_CEN_LAT:   39.34594
    ...             ...
    CEN_LAT:        39.34594
    day_of_week:    weekday
    sector:         AG
    year:           2021
    month:          1
    day_type:       weekdy